In [0]:
%run ../EcommerceProject/enrich_table

# Read Tables

In [0]:
dim_customers = spark.read.table("ecommerceproject.enriched.dim_customers")
dim_products = spark.read.table("ecommerceproject.enriched.dim_products")
orders_raw = spark.read.table("ecommerceproject.default.raw_orders")

# Enrich Fact Orders with Dimensions

In [0]:
dim_products_canonical = (
    dim_products
    .withColumn(
        "rn",
        row_number().over(
            Window.partitionBy("product_id")
                  .orderBy(col("effective_start_date").desc())
        )
    )
    .filter(col("rn") == 1)
    .drop("rn")
)
logger.info("identified {} canonical products".format(dim_products_canonical.count()))

fact_orders_enriched = (
    orders_raw.alias("o")
    .join(
        dim_customers.filter(col("is_current") == True).alias("c"),
        col("o.customer_id") == col("c.customer_id"),
        "left"
    )
    .join(
        dim_products_canonical.filter(col("is_current") == True).alias("p"),
        col("o.product_id") == col("p.product_id"),
        "left"
    )
    .withColumn("order_date", to_date(col("o.order_date"), "d/M/yyyy"))
    .withColumn("ship_date", to_date(col("o.ship_date"), "d/M/yyyy"))
    .withColumn("order_year", year(col("order_date")))
    .withColumn("profit", round(col("o.profit"), 2))
    .select(
        col("o.order_id"),
        col("order_date"),
        col("order_year"),
        col("ship_date"),
        col("o.ship_mode"),
        col("o.row_id"),
        col("o.customer_id"),
        col("c.customer_name"),
        col("c.country"),
        col("o.product_id"),
        col("p.category"),
        col("p.subcategory").alias("sub_category"),
        col("o.quantity"),
        col("o.price"),
        col("o.discount"),
        col("profit")
    )
)
try:
    fact_orders_enriched.write \
        .format("delta") \
        .mode("overwrite") \
        .option("overwriteSchema", "true") \
        .saveAsTable("ecommerceproject.enriched.orders_enriched")
except Exception as e:
    logger.error("Failed to write orders_enriched Delta table", exc_info=True)
    raise
logger.info("enriched orders written to delta table : {}".format(fact_orders_enriched.count()))


In [0]:
if spark.catalog.tableExists("ecommerceproject.enriched.orders_enriched"):
    spark.sql("""
    ALTER TABLE ecommerceproject.enriched.orders_enriched
    SET TBLPROPERTIES (
      delta.autoOptimize.optimizeWrite = true,
      delta.autoOptimize.autoCompact = true
    )
    """)
logger.info("enriched orders table optimized")

# Aggregated Profit Product Category, Year, Customer, Sub Product Category

In [0]:
agg_profit_df= fact_orders_enriched.groupBy("order_year","category","sub_category","customer_name").agg(round(sum("profit"),2).alias("total_profit"))
try:
    agg_profit_df.write \
        .format("delta") \
        .mode("overwrite") \
        .saveAsTable("ecommerceproject.analytics.agg_profit")
except Exception as e:
    logger.error("Failed to write agg_profit Delta table", exc_info=True)
    raise

## Profit By Year

In [0]:
%sql
select order_year,round(sum(profit),2) as total_profit
from ecommerceproject.enriched.orders_enriched
group by order_year
order by order_year desc

## Profit by Year + Product Category

In [0]:
%sql
select order_year,category,round(sum(profit),2) as total_profit
from ecommerceproject.enriched.orders_enriched
where category is not null
group by order_year,category
order by order_year desc

## Profit by Customer

In [0]:
%sql
select customer_id, round(sum(profit),2) as total_profit 
from ecommerceproject.enriched.orders_enriched
group by customer_id
order by total_profit asc

In [0]:
%sql
select order_year,customer_id, round(sum(profit),2) as total_profit 
from ecommerceproject.enriched.orders_enriched
group by order_year,customer_id
order by order_year,total_profit desc